---
# 11. Strong Connected Component

|Problem|Dfficulty|Link|
|--------|--------|-----------|
|1489. Find Critical and Pseudo-Critical Edges in Minimum Spanning Tree | <span style="color:red">Hard</span> | https://leetcode.com/problems/find-critical-and-pseudo-critical-edges-in-minimum-spanning-tree/description |
|1568. Minimum Number of Days to Disconnect Island |  <span style="color:red">Hard</span> | https://leetcode.com/problems/minimum-number-of-days-to-disconnect-island/description |

## 1489. Find Critical and Pseudo-Critical Edges in Minimum

# Intuition
To find the critical and pseudo-critical edges in a minimum spanning tree (MST), we can use Kruskal's algorithm for MST construction. 

# Approach
1. **Union-Find Structure**:
   - Implement union-find to keep track of connected components.
   - Use path compression and union by rank to optimize the union-find operations.

2. **MST Calculation**:
   - Define a function `mst` that calculates the weight of the MST while optionally including or excluding specific edges.
   - Use the union-find structure to construct the MST by iterating over the sorted edges.

3. **Identifying Critical and Pseudo-Critical Edges**:
   - Calculate the weight of the original MST without any edge constraints.
   - For each edge, calculate the MST weight excluding that edge. 
        - If this weight is greater than the original MST weight, the edge is critical.
   - For each edge, calculate the MST weight including that edge from the start. 
        - If this weight equals the original MST weight, the edge is pseudo-critical.

# Complexity

## Time complexity: `O(E^2 log E)`, where `E` is the number of edges.

## Space complexity: `O(V + E)`, where `V` is the number of vertices and `E` is the number of edges. 

```python
class Solution :
    def find(self,u,parent):
        if u==parent[u]:
            return u
        return self.find(parent[u],parent)

    def unionDSU(self,u,v,parent) :
        p1=self.find(u,parent)
        p2=self.find(v,parent)
        parent[p2]=p1

    def mst(self,k,edges,includeEdge,excludeEdge) :
        n=len(includeEdge)
        m=len(excludeEdge)
        parent=[]
        for i in range(k): 
            parent.append(i);
        
        res=0
        count=0

        if n!= 0 :
            self.unionDSU(includeEdge[0], includeEdge[1], parent)
            res+=includeEdge[2]
            count+=1
        
        for edge in edges:
            u=edge[0];
            v=edge[1];
            cost=edge[2];

            if m!=0 and excludeEdge[0]==u and excludeEdge[1]==v and excludeEdge[2]==cost :
                continue
            
            if n!=0 and includeEdge[0]==u and includeEdge[1]==v and includeEdge[2]==cost :
                continue
            
            p1=self.find(u,parent)
            p2=self.find(v,parent)

            if p1!= p2:
                self.unionDSU(p1, p2, parent)
                res+= cost
                count += 1
                    
        return  res if count==k-1 else float('inf')
    
    def findCriticalAndPseudoCriticalEdges(self, k:int, edges:List[List[int]]) -> List[List[int]]:
        
        originalEdges=[]
        for edge in edges : 
            originalEdge=[edge[0],edge[1],edge[2]]
            originalEdges.append(originalEdge)
            
        X=len(originalEdges)
        ans=[]
        criticalEdges=[]
        pseudoCriticalEdges=[]

        
        edges=sorted(edges, key = lambda x: x[2])

        emptyVector=[]
        originalCost = self.mst(k, edges, emptyVector, emptyVector)
        
        for i in range(X): 
            excludedCost = self.mst(k, edges, emptyVector, originalEdges[i])
            includedCost = self.mst(k, edges, originalEdges[i], emptyVector)

            if excludedCost > originalCost :
                criticalEdges.append(i)
                
            elif includedCost==originalCost : 
                pseudoCriticalEdges.append(i)
                    
        ans.append(criticalEdges)
        ans.append(pseudoCriticalEdges)
        return ans
```

---
## 1568. Minimum Number of Days to Disconnect Island

# Intuition
The problem can be approached by checking the connectivity of the grid and then attempting to disconnect the island by removing one or two cells.

# Approach
1. **DFS for Island Counting**:
   - Implement a DFS function to traverse and mark cells of an island.
   - Use this function to count the number of islands in the grid.

2. **Check Connectivity**:
   - Implement a function to check if the grid is connected (i.e., it has exactly one island).

3. **Minimum Days Calculation**:
   - If the grid is initially disconnected or has only one land cell, return 0 days.
   - Try removing each land cell one by one and check if the grid becomes disconnected.
   - If removing one cell disconnects the grid, return 1 day.
   - If no single cell removal disconnects the grid, it means at least 2 removals are needed to disconnect the island.

# Complexity

## Time complexity: `O(n * m * (n + m))`, where `n` is the number of rows and `m` is the number of columns. 
## Space complexity: `O(n * m)` for the grid and visited markers during DFS.


```cpp
typedef pair<int, int> pii;

class Solution {
    vector<pii> directions{{1, 0}, {-1, 0}, {0, 1}, {0, -1}}; // 4-directional vectors
    
    // Function to count islands using DFS
    int dfs(vector<vector<int>>& grid, int x, int y) {
        if (x < 0 || y < 0 || x >= grid.size() || y >= grid[0].size() || grid[x][y] != 1) return 0;
        grid[x][y] = -1; // Mark as visited
        for (auto& d : directions) dfs(grid, x + d.first, y + d.second);
        return 1;
    }

    // Function to check if the grid is connected
    bool isConnected(vector<vector<int>> grid) {
        int count = 0;
        for (int i = 0; i < grid.size(); ++i) {
            for (int j = 0; j < grid[0].size(); ++j) {
                if (grid[i][j] == 1) {
                    count += dfs(grid, i, j);
                    if (count > 1) return false; // More than one island found
                }
            }
        }
        return count == 1; // True if exactly one island is found
    }

public:
    int minDays(vector<vector<int>>& grid) {
        // If grid is initially disconnected or has only one cell that is land, no days needed or 1 day needed
        if (!isConnected(grid) || grid.size() * grid[0].size() == 1) return 0;
        
        for (int i = 0; i < grid.size(); ++i) {
            for (int j = 0; j < grid[0].size(); ++j) {
                if (grid[i][j] == 1) {
                    grid[i][j] = 0; // Try removing the cell
                    if (!isConnected(grid)) return 1; // If disconnected by removing one cell, 1 day needed
                    grid[i][j] = 1; // Revert the change
                }
            }
        }
        
        // If no single removal disconnects the grid, it means 2 removals are needed, or it's a special case like a loop
        return 2;
    }
};
```